In [2]:
!pip install map-boxes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import sys
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import random
import torch
import torch.nn as nn
import cv2

/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /usr/local/lib/python3.7/dist-packages/map_boxes/compute_overlap.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

In [ ]:
'''T = [
                A.Blur(p=0.07),
                A.MedianBlur(p=0.01),
                A.ToGray(p=0.05),
                A.CLAHE(always_apply=False, p=0.05, clip_limit=(1, 16), tile_grid_size=(11, 8)),
                A.RandomBrightnessContrast(always_apply=False, p=0.1, brightness_limit=(-0.3, -0.1), contrast_limit=(-0.2, 0.2), brightness_by_max=True),
                A.RandomGamma(p=0.05),
                A.ImageCompression(quality_lower=75, p=0.02),
                ToSepia(always_apply=False, p=0.05)]'''

In [ ]:
!pip install -r /content/yolov5/requirements.txt

In [ ]:
#в папке data_for_yolo создадим dataset.yaml со следующим содержимым

yaml_content = """
train: /content/data_for_yolo/data/images/train/
val: /content/data_for_yolo/data/images/test/

# number of classes
nc: 1

# class names
names: ['human']
"""

In [6]:
path_to_data = '/content/drive/MyDrive/tatarstan_yaml/face.yaml'

In [ ]:
!nvidia-smi

In [ ]:
!python ./yolov5/train.py --img 640 --batch 16 --epochs 50 --data {path_to_data} --weights /content/drive/MyDrive/weights/face_s640_0.3.pt --name head_results --cache

In [ ]:
!python3 yolov5/detect.py --weights /content/yolov5/runs/train/head_results4/weights/best.pt --img 1220 --conf 0.325 --source /content/drive/MyDrive/data_for_yolo/data/images/test/face --save-txt --save-conf

In [1]:

def get_soliton_labels_df(path_to_txt_folder):
  simple_solution = []
  for detection_file in os.listdir(path_to_txt_folder):
    img_name = detection_file.split('.')[0] + '.jpg'
    with open(path_to_txt_folder + detection_file, 'r') as f:
      data = f.read()
      data = [i for i in data.split('\n') if i != '']
    for line in data:
      
      val = [float(i) for i in line.split()]

      cls, xywh, conf = val[0], val[1:5], val[5]
      center_x, center_y, width, height = xywh
      xmin = center_x - (width / 2)
      xmax = center_x + (width / 2)
      ymin = center_y - (height / 2)
      ymax = center_y + (height / 2)
      simple_solution.append([img_name, cls, conf, xmin, xmax, ymin, ymax])
  return simple_solution

In [40]:
simple_solution = get_soliton_labels_df('yolov5/runs/detect/exp5/labels/')
simple_solution = pd.DataFrame(simple_solution, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])

In [41]:
def get_test_labels_df(path_to_txt_folder):
  simple_solution = []
  for detection_file in os.listdir(path_to_txt_folder):
    img_name = detection_file.split('.')[0] + '.jpg'
    with open(path_to_txt_folder + detection_file, 'r') as f:
      data = f.read()
      data = [i for i in data.split('\n') if i != '']
    for line in data:
      val = [float(i) for i in line.split()]
      cls, center_x, center_y, width, height = val
      xmin = center_x - (width / 2)
      xmax = center_x + (width / 2)
      ymin = center_y - (height / 2)
      ymax = center_y + (height / 2)
      simple_solution.append([img_name, cls, xmin, xmax, ymin, ymax])
  return simple_solution

In [42]:
test_labels = get_test_labels_df('/content/drive/MyDrive/data_for_yolo/data/labels/test/face/')
test_labels = pd.DataFrame(test_labels, columns=['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax'])

In [43]:
mean_ap, average_precisions = mean_average_precision_for_boxes(test_labels, simple_solution, iou_threshold=0.325, verbose=False)

In [44]:
mean_ap

0.5610231396497323

In [ ]:
#/content/participants/test/images

In [ ]:

!python3 yolov5/detect.py --weights /content/drive/MyDrive/weights/face_s640_0.3.pt --img 1220 --conf 0.325 --source /content/drive/MyDrive/test_dataset_test/test/images/ --save-txt --save-conf

In [32]:
def get_soliton_labels_df_simple_solution(path_to_txt_folder):
  simple_solution = []
  for detection_file in os.listdir(path_to_txt_folder):
    img_name = detection_file.split('.')[0] + '.jpg'
    with open(path_to_txt_folder + detection_file, 'r') as f:
      data = f.read()
      data = [i for i in data.split('\n') if i != '']
    for line in data:
      val = [float(i) for i in line.split()]
      _, xywh, conf = val[0], val[1:5], val[5]
      center_x, center_y, width, height = xywh
      xmin = center_x - (width / 2)
      xmax = center_x + (width / 2)
      ymin = center_y - (height / 2)
      ymax = center_y + (height / 2)
      simple_solution.append([img_name, 2, conf, xmin, xmax, ymin, ymax])
  return simple_solution

In [33]:
simple_solution = get_soliton_labels_df_simple_solution('yolov5/runs/detect/exp4/labels/')
simple_solution = pd.DataFrame(simple_solution, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])

In [ ]:
simple_solution

In [ ]:
df = pd.read_csv("car.csv", sep=';', index_col=None)

In [ ]:
df